In [ ]:
!pip install gdown
!gdown --id 1siZtGusB1LfQVapTvNOiYi8aeKKAgcDF
!unzip full-shape.zip -d /content/

In [ ]:
import pickle

# Load training data
with open('full_shape_train_data.pkl', 'rb') as train_file:
    train_data = pickle.load(train_file)

# Print the structure of the first object in the dataset
print("Keys in the first object:", train_data[0].keys())
print("Semantic Class:", train_data[0]['semantic class'])
print("Affordances:", train_data[0]['affordance'])
print("Coordinate Shape:", train_data[0]['full_shape']['coordinate'].shape)
print("Label Keys:", train_data[0]['full_shape']['label'].keys())


In [ ]:
from collections import defaultdict

affordance_counts = defaultdict(lambda: defaultdict(int))

for entry in train_data:
    semantic_class = entry['semantic class']
    for affordance, label in entry['full_shape']['label'].items():
        if label.max() > 0:  # Check if there are non-zero labels
            affordance_counts[semantic_class][affordance] += 1

for semantic_class, counts in affordance_counts.items():
    print(f"{semantic_class}:")
    for affordance, count in counts.items():
        print(f"  {affordance}: {count} objects")


In [ ]:
import matplotlib.pyplot as plt

affordance_totals = defaultdict(int)
for semantic_class, counts in affordance_counts.items():
    for affordance, count in counts.items():
        affordance_totals[affordance] += count

# Bar plot
plt.bar(affordance_totals.keys(), affordance_totals.values())
plt.xticks(rotation=45, ha='right')
plt.title("Frequency of Affordances Across All Classes")
plt.ylabel("Number of Objects")
plt.show()


In [ ]:
import pandas as pd

class_affordance_matrix = pd.DataFrame.from_dict(affordance_counts, orient='index').fillna(0)
print(class_affordance_matrix)

# Save to a CSV for further analysis
class_affordance_matrix.to_csv("class_affordance_matrix.csv")

In [ ]:
# Select the first object
object_data = val_data[0]

# Extract relevant information
coords = object_data['full_shape']['coordinate']
affordance_label = object_data['full_shape']['label']['grasp']  # Replace 'grasp' with desired affordance

print("Selected Object Class:", object_data['semantic class'])
print("Affordance Label Shape:", affordance_label.shape)

In [ ]:
import numpy as np

# Convert to numpy arrays for easy handling
coords = np.array(coords)  # Shape: [N, 3]
affordance_label = np.array(affordance_label)  # Shape: [N]

# Print summary of the affordance labels
print("Unique Label Values:", np.unique(affordance_label, return_counts=True))


In [ ]:
print("Coordinate Range:")
print("X:", coords[:, 0].min(), "-", coords[:, 0].max())
print("Y:", coords[:, 1].min(), "-", coords[:, 1].max())
print("Z:", coords[:, 2].min(), "-", coords[:, 2].max())

print("\nAffordance Label Range:")
print("Min:", affordance_label.min(), "Max:", affordance_label.max())

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def plot_3d_points(coords, labels, title="3D Scatter Plot"):
    fig = plt.figure(figsize=(10, 7))
    ax = fig.add_subplot(111, projection='3d')
    scatter = ax.scatter(
        coords[:, 0], coords[:, 1], coords[:, 2],
        c=labels, cmap='coolwarm', s=10
    )
    plt.colorbar(scatter)
    ax.set_title(title)
    plt.show()

# Plot the object with ground truth labels
plot_3d_points(coords, affordance_label, title=f"{object_data['semantic class']} - Pull")


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Select an object with non-zero affordance labels
selected_class = "Door"
selected_affordance = "Pull"

# Find an object with non-zero labels for the selected affordance
for entry in train_data:
    if entry["semantic class"] == selected_class and selected_affordance in entry["full_shape"]["label"]:
        label = entry["full_shape"]["label"][selected_affordance]
        if label.max() > 0:  # Ensure non-zero values
            coords = entry["full_shape"]["coordinate"]
            break

# Plot the coordinates and affordance label
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

# Normalize the label values for better visualization
norm_labels = label / label.max()

# Scatter plot with color-coded affordance regions
sc = ax.scatter(
    coords[:, 0], coords[:, 1], coords[:, 2],
    c=norm_labels, cmap="coolwarm", s=2
)

ax.set_title(f"{selected_class} - {selected_affordance}")
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")

# Add a color bar to show the affordance intensity
plt.colorbar(sc, label="Affordance Intensity")
plt.show()
